# Basic M3C2-EPalgorithm

This presents how the M3C2-EP algorithm ([Winiwarter et al., 2013](#References)) for point cloud distance computation can be run using the `py4dgeo` package. 

As a first step, we import the `py4dgeo` and `numpy` packages:

In [ ]:
import numpy as np
import py4dgeo

For this demonstrator notebook, we download test data from the [py4dgeo test data repository](https://github.com/3dgeo-heidelberg/py4dgeo-test-data/). This is not required if you are working with your own data:

In [ ]:
py4dgeo.ensure_test_data_availability()

Next, we need to load two datasets that cover the same scene at two different points in time. Point cloud datasets are represented by `numpy` arrays of shape `n x 3` using a 64 bit floating point type (`np.float64`).

Please ensure two datasets include scan positions which are specified by attribute name `sp_name` and scan positions configuration information in `sp_file`. 

Here, we work with a rather small synthetical data set:

In [ ]:
epoch1, epoch2 = py4dgeo.read_from_las("t1.laz", "t2.laz", sp_name="point_source_id", sp_file="sps.json")

The analysis of point cloud distances is executed on so-called *core points* (cf. Lague et al., 2013). These could be, e.g., one of the input point clouds, a subsampled version thereof, points in an equidistant grid, etc. Here, we choose a subsampling by taking every 50th point of the reference point cloud:

In [ ]:
corepoints = epoch1.cloud[::50]

The algorithm needs covariance matrix of shape `12 x 12`, affine transformation matrix of shape `3 x 4` and translation vector (3 parameters).

In [ ]:
Cxx = np.loadtxt('./test_data/Cxx.csv', dtype=np.float64, delimiter=',')
tfM = np.loadtxt('./test_data/tfM.csv', dtype=np.float64, delimiter=',')
redPoint = np.loadtxt('./test_data/redPoint.csv', dtype=np.float64, delimiter=',')

Next, we instantiate the algorithm class and run the distance calculation:

In [ ]:
m3c2_ep = py4dgeo.M3C2EP(
    epochs=(epoch1, epoch2),
    corepoints=corepoints,
    normal_radii=(0.5, 1.0, 2.0),
    cyl_radii=(0.5,),
    max_distance=3.0,
    Cxx=Cxx,
    tfM=tfM,
    refPointMov=redPoint
)

In [ ]:
distances, uncertainties, covariance = m3c2_ep.run()

The calculated result is an array  with one distance per core point. The order of distances corresponds exactly to the order of input core points.

In [ ]:
distances

Corresponding to the derived distances, an uncertainty array is returned which contains several quantities that can be accessed individually: The level of detection `lodetection`, the spread of the distance across points in either cloud (`spread1` and `spread2`, by default measured as the standard deviation of distances) and the total number of points taken into consideration in either cloud (`num_samples1` and `num_samples2`):

In [ ]:
uncertainties["lodetection"]

In [ ]:
uncertainties["spread1"]

In [ ]:
uncertainties["num_samples1"]

Corresponding to the derived distances, a 3D covariance information for the point cloud is returned.

In [ ]:
covariance['cov1'].shape

In [ ]:
covariance['cov1'][0,:,:]

Finally we could visualize our distances results.

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
def plt_3d(corepoints, distances):
    fig, ax = plt.subplots(figsize=(10,10), subplot_kw={'projection' : '3d'})

    # add axis labels
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    
    # plot the corepoints colored by their distance
    x,y,z = np.transpose(corepoints)
    pts = ax.scatter(x,y,z,s=10, c=distances, vmin=-1, vmax=1, cmap=cm.seismic_r)
    
    # add colorbar
    cmap = plt.colorbar(pts, shrink=0.5, label="Distance [m]", ax=ax)  
    
    # add title
    ax.set_title("Visualize Changes")
    
    ax.set_aspect('equal')
    ax.view_init(22,113)
    plt.show()

In [ ]:
plt_3d(corepoints, distances)

### References
* Winiwarter, L., Anders, K., & Höfle, B. (2021). M3C2-EP: Pushing the limits of 3D topographic point cloud change detection by error propagation. ISPRS Journal of Photogrammetry and Remote Sensing, 178, 240-258. doi: [10.1016/j.isprsjprs.2021.06.011](https://doi.org/10.1016/j.isprsjprs.2021.06.011).

* Lague, D., Brodu, N., & Leroux, J. (2013). Accurate 3D comparison of complex topography with terrestrial laser scanner: Application to the Rangitikei canyon (N-Z). ISPRS Journal of Photogrammetry and Remote Sensing, 82, pp. 10-26. doi: [10.1016/j.isprsjprs.2013.04.009](https://doi.org/10.1016/j.isprsjprs.2013.04.009).